# Magic: The Gathering Market Analysis 
* Physical Cards, English Only, Retail Pricing

In [ ]:
import pandas as pd

In [ ]:
dfm = pd.read_csv('../data/dataMagic/cardsMagic.csv') # 8/26/25
# see all columns
pd.set_option('display.max_columns', None)

In [ ]:
dfm.shape

* There are a ton of missing values, but that makes sense because some card types have no integer values (e.g. sorceries don't have creature power).
* Determining columns we want to keep in this initial data frame  

In [ ]:
# For a full ist of collumns (most of these probably can be dropped for our purposes)
dfm.columns

In [ ]:
# Specify the needed columns
dfm = dfm[["availability", "colors", "language", "name", "rarity", "setCode", "types", "uuid"]]

dfm

In [ ]:
# identify NA and Null values for the newly specified df columns
dfm.isna().sum()

In [ ]:
dfm.loc[dfm['colors'].isna()]

* There are cards that are inherently "colorless" in MTG. It's interesting that they are not labeled as such in this dataset and are NaN instead.  
* Checking to make sure those are all listed as "NaN" instead of a different designation.  
* B, G, R, U, W are all valid color codes (cards can be multiple colors).
* Colors will be useful for our functions later but not necessarily useful for determining card values.

In [ ]:
dfm["colors"].unique()

In [ ]:
dfm.loc[dfm["colors"].isna(), "types"].unique()

In [ ]:
# Replace all NaN with "C" for "Colorless"
dfm["colors"] = dfm["colors"].fillna("C")

In [ ]:
# Check to see if it worked
dfm.loc[dfm["colors"].isna(), "types"].unique()

In [ ]:
# Second check
dfm["colors"].unique()

* We are strictly looking at physical cards (paper) and not cards that are solely available for online play.  
    - This is due to the wild disparity between online and physical cards.  
    - We are looking only to gauge secondary, physical market.  

In [ ]:
dfm[dfm["availability"] == "mtgo"]

* Having digital and physical cards together in this dataset also accounts for why we're seeing so many rows with the same names.

In [ ]:
dfm.duplicated(subset=["name"], keep=False)

In [ ]:
dfm.value_counts('availability')

In [ ]:
# Specifying the rows to keep involving paper
dfm = dfm[
    (dfm["availability"] == "mtgo, paper") | 
    (dfm["availability"] == "paper") | 
    (dfm["availability"] == "arena, mtgo, paper") |
    (dfm["availability"] == "arena, paper") 
    ]

dfm

In [ ]:
# Check to make sure it worked as intended
dfm.value_counts("availability")

* There are still rows with the same card name, but those are alt arts, promos, etc. each with their own values, and have separate uuid #s.

In [ ]:
# Check for duplicated rows to be sure (there are none)
dfm.duplicated(keep=False).sum()

* We're only going to be working with the english version of cards

In [ ]:
dfm = dfm[dfm["language"] == "English"]

* We're also going to remove the basic lands from each set.  
* These lands are printed every set in bulk and are mostly worthless, barring certain outliers.  
* This will tighten our dataset and focus it toward value.  

In [ ]:
basic_lands = ["Forest", "Island", "Mountain", "Plains", "Swamp"]

# the tilde flips the boolean values, so we are keeping what is NOT IN basic_lands
dfm = dfm[~dfm["name"].isin(basic_lands)]

In [ ]:
# Check to make sure it worked correctly
dfm[dfm["name"] == "Forest"]

* Documenting set code info for reference later  

In [ ]:
dfm["setCode"].nunique()

In [ ]:
dfm["setCode"].unique()

* Now that we have a clean dataset, let's join in set details from another CSV file.  
* We're looking to add set name, release date, and release year with this join.

In [ ]:
dfmSets = pd.read_csv('../data/dataMagic/setsMagic.csv') # 9/22/25
dfmSets.info()

In [ ]:
dfmSets.isna().sum()

In [ ]:
dfm2 = pd.merge(dfm, dfmSets, on = "setCode", how = "inner")
dfm2.head()

* Now let's bring in the card prices.

In [ ]:
dfmPrices =  pd.read_csv('../data/dataMagic/pricesMagic.csv') # 8/27/25
dfmPrices.head()

* We again only want paper prices.  
* We also don't want buylist prices either, only retail.  
* CardMarket is strictly a EU trader, so their prices are in EUR.  

In [ ]:
dfmPrices["gameAvailability"].unique()

In [ ]:
dfmPrices["providerListing"].unique()

In [ ]:
dfmPrices["priceProvider"].unique()

In [ ]:
mtgo = ["mtgo"]
buylist = ["buylist"]
cardmarket = ["cardmarket"]

dfmPrices = dfmPrices[~dfmPrices["gameAvailability"].isin(mtgo)]
dfmPrices = dfmPrices[~dfmPrices["providerListing"].isin(buylist)]
dfmPrices = dfmPrices[~dfmPrices["priceProvider"].isin(cardmarket)]
dfmPrices

In [ ]:
dfm3 = pd.merge(dfm2, dfmPrices, on = "uuid", how = "left")
dfm3.head(5)

In [ ]:
dfmPrices.isna().sum()

In [ ]:
dfm3.isna().sum()

In [ ]:
dfm3.loc[dfm3['cardFinish'].isna()]

* The goal now is to add an average price for all cards, calculated from the price providers.

In [ ]:
# using transform to add a column with the average price
dfm3["avgMarketPrice"] = dfm3.groupby(['uuid', 'cardFinish'])['price'].transform('mean')
dfm3.head()

In [ ]:
# Round avgPrice to 2 decimals
dfm3["avgMarketPrice"] = dfm3["avgMarketPrice"].round(2)
dfm3.head()

* Now to drop card price column and to make one row per uuid, so we don't skew results later one with having multiple indexes per card and price source.  
* There will still be multiple rows for some card names, but they will have different uuids for alt arts and promos.  
* Availability column is also no longer needed now that we have gameAvailability.  
* cardFinish is no longer needed since it is now part of the average price.  
* priceProvider is no longer needed as well, which will help us de-dupe.  

In [ ]:
# check to make sure for no NA values after merges
dfm3.isna().sum()

In [ ]:
dfm3.dtypes

* We need to convert releaseDate to datetime so it will sort correctly.

In [ ]:
# We don't need availability any longer, since we now have gameAvailability from pricesMagic.csv
dfm3.drop(columns = ["availability"], inplace = True)
dfm3.head()

In [ ]:
dfm3['releaseDate'] = pd.to_datetime(dfm3['releaseDate'], format = '%m/%d/%Y', errors = 'raise')
dfm3.dtypes

In [ ]:
# A more viewer-friendly order
newOrderM = ['name', 'setCode', 'setName', 'language', 'types', 'colors', 'rarity', 'cardFinish', 'releaseDate', 'releaseYear', 'gameAvailability', 
             'priceProvider', 'price', 'avgMarketPrice', 'currency', 'providerListing', 'date', 'uuid']

dfm3 = dfm3[newOrderM]
dfm3.head()

In [ ]:
# Order by year, then setName, then name
dfm3 = dfm3.sort_values(by=["releaseDate", "setName", "name"])
dfm3.head()

In [ ]:
# Reset index after manipulation and to check new number of rows
# Dropping the original index column
dfm3 = dfm3.reset_index(drop = True)
dfm3.head()

In [ ]:
#dfm3.to_csv("../data/dataMagic/completeMagicIndPrices.csv", index = False)

In [ ]:
# dfm3 will be used for SQL queries and individual price lookups
# dfm4 will be used for visualization, based on avgMarketPrice, and will be cleaner with less indexes based on UUID
dfm4 = dfm3.drop(columns = ["price", "priceProvider"])
dfm4.head()

In [ ]:
dfm4.drop_duplicates(keep = "first", inplace = True)
dfm4.head()

In [ ]:
# Reset index again
dfm4 = dfm4.reset_index(drop = True)
dfm4.head()

In [ ]:
#dfm4.to_csv("../data/dataMagic/completeMagicClean.csv", index = False)

* We started with ~106k rows x 78 columns and now ~89k rows x 15 columns.

In [ ]:
# Checking name lookup for reference
dfm4[dfm4["name"] == "Black Lotus"]

* Prep for visualizations

In [ ]:
setValueMagic = dfm4.groupby("setName").agg({
    "avgMarketPrice" : "sum",
    "releaseYear" : "first"
})

# reset index so setName and releaseYear are columns
setValueMagicClean = setValueMagic.reset_index()
# ordering
setValueMagicClean = setValueMagicClean.sort_values(by=["releaseYear", "avgMarketPrice"], ascending = False)
# outputting
setValueMagicClean.to_csv("../data/dataMagic/setValueMagicClean.csv", index = False)

In [ ]:
# To show the top sets from each year for graph ax text
setSumMagic = dfm4.groupby(["releaseYear", "setName"], as_index=False)["avgMarketPrice"].sum()

# For each year, the set with the highest total price
topSetsMagic = setSumMagic.loc[setSumMagic.groupby("releaseYear")["avgMarketPrice"].idxmax()]
topSetsMagic = topSetsMagic.sort_values("avgMarketPrice").round()
topSetsMagic = topSetsMagic.sort_values("releaseYear")

topSetsMagic